In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/02/28 17:24:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pyspark.__version__

'3.3.2'

In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-02-28 05:09:09--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.213, 52.84.160.73, 52.84.160.116, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M   185MB/s    in 0.3s    

2025-02-28 05:09:09 (185 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [8]:
df \
    .repartition(4) \
    .write.parquet(f'yellow/2024/10/')

In [77]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [4]:
df = spark.read.parquet('yellow/2024/10')

In [9]:
df = df \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [10]:
df.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [11]:
common_schema = []

for col in df.columns:
    common_schema.append(col)

In [12]:
common_schema 

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [45]:
df.count()

3833771

In [15]:
!ls -lh yellow/2024/10

total 97M
-rw-r--r-- 1 ritachen ritachen   0 Feb 28 06:32 _SUCCESS
-rw-r--r-- 1 ritachen ritachen 25M Feb 28 06:32 part-00000-576d0436-06a6-4ada-9294-6907696b77b5-c000.snappy.parquet
-rw-r--r-- 1 ritachen ritachen 25M Feb 28 06:32 part-00001-576d0436-06a6-4ada-9294-6907696b77b5-c000.snappy.parquet
-rw-r--r-- 1 ritachen ritachen 25M Feb 28 06:32 part-00002-576d0436-06a6-4ada-9294-6907696b77b5-c000.snappy.parquet
-rw-r--r-- 1 ritachen ritachen 25M Feb 28 06:32 part-00003-576d0436-06a6-4ada-9294-6907696b77b5-c000.snappy.parquet


In [16]:
df.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [19]:
df.select('VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count') \
.filter('pickup_datetime'

DataFrame[VendorID: int, pickup_datetime: timestamp, dropoff_datetime: timestamp, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [21]:
df.createOrReplaceTempView('trip_data')

In [30]:
spark.sql("""
SELECT
    count(1)
FROM 
    trip_data
WHERE 
    date(pickup_datetime) = '2024-10-15'

""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [34]:
spark.sql("""
SELECT
    (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime))/3600 as length_of_trip_seconds
FROM 
    trip_data
ORDER BY 
    length_of_trip_seconds DESC
LIMIT 1
""").show()

+----------------------+
|length_of_trip_seconds|
+----------------------+
|    162.61777777777777|
+----------------------+



In [37]:
df_zones = spark.read.parquet('zones/')

In [38]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [57]:
df_zones_freq = spark.sql("""
SELECT 
    PULocationID AS zoneID, 
    COUNT(1) AS cnt
FROM trip_data
GROUP BY 
    1
""")

In [58]:
df_result = df_zones_freq.join(df_zones, df_zones_freq.zoneID == df_zones.LocationID)

In [59]:
df_zone_counts = df_result \
    .groupBy('Zone') \
    .count() \
    .orderBy('count') \
    .limit(1)

df_zone_counts.show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+

